In [1]:
# We import covalent and the plugin
import covalent as ct
import covalent_slurm_plugin as ct_sl

# just to test some numpy computation
import numpy as np


In [6]:
# this is an example of using slurm plugin to launch jobs on meluxina
# we create this dictionary, which contains the login credentials and slurm job parameters.
slurm_dic = {
    'username': 'username',
    'address': 'login.lxp.lu',
    'ssh_key_file': '~/.ssh/id_ed25519',
    'cert_file': None,
    'ssh_port': 8822,
    'remote_workdir': '/project/home/account-d/workdir-covalent', # a directory where you have access to read and write
    'create_unique_workdir': False,
    'options': {'account': 'account-id', # given to project,
                'partition': 'cpu',
                'qos': 'default',
                'nodes': 1,
                'time': '00:15:00',
                'ntasks': 128
                },
    'variables': {'OMP_NUM_THREADS': 128},
    'conda_env': 'wf',
    'prerun_commands': '~/bin/micromamba activate wf',
    'postrun_commands': None,
    'use_srun': None,
    'srun_options': {"n": 1, "c": 8},
    'srun_append': None,
    'bashrc_path': None,
    'slurm_path': None,
    'poll_freq': None,
    'cleanup': False,
    'cache_dir': None,
    'log_stdout': './out',
    'log_stderr': './err',
    'time_limit': -1,
    'retries': 3,
    'ignore_versions': None
}

# here we instantiate the slurm executor with the above dictionary
slurm = ct_sl.SlurmExecutor(**slurm_dic)
local=ct.executor.LocalExecutor()

@ct.electron(executor=slurm)
def get_eig(mats):
    import socket
    host = socket.gethostname()
    import numpy as np
    es = [np.linalg.eigvalsh(mat) for mat in mats]
    return (es, host)

@ct.electron(executor=local)
def get_matrix(n=1000):
    import numpy as np
    mat = np.random.random((n, n))
    mat = 2 * mat - 1
    mat = mat + mat.T
    mat *= 0.5
    return mat


@ct.lattice(executor=local)
def gather_eigs(num=128, n=1000):
    #e = np.zeros((num, n))
    #hosts = []
    mats = [get_matrix(n) for i in range(num)]
    res = get_eig(mats)
    return res

run_id = ct.dispatch(gather_eigs)(num=12, n=2000)
output = ct.get_result(run_id, wait=True)
#print(e.min(), e.max())
print(output.status)
print(output.result)


FAILED
None


The above says failed, and this is okay, as we did not put the real project ids, and slurm could not be submitted.

In [4]:
output.result

[(array([-36.76891646, -36.05256116, -36.01414272, ...,  35.97082165,
          36.01665094,  36.48772427]),
  'mel0341'),
 (array([-36.53619195, -36.16779173, -36.0614137 , ...,  35.86567811,
          36.10875373,  36.26276746]),
  'mel0014'),
 (array([-36.43418594, -36.17718831, -35.94032391, ...,  35.73770268,
          35.97083045,  36.0372198 ]),
  'mel0439'),
 (array([-36.4341718 , -36.14836625, -36.08860462, ...,  35.84654148,
          36.28884679,  36.38385895]),
  'mel0317')]

In [4]:
for i in output.get_all_node_results():
    print(i)

{'node_id': 0, 'node_name': 'get_eig', 'start_time': datetime.datetime(2024, 10, 25, 11, 32, 38, 31536), 'end_time': datetime.datetime(2024, 10, 25, 11, 36, 47, 796387), 'status': Status(STATUS=<StatusEnum.COMPLETED: 'COMPLETED'>), 'output': <covalent.TransportableObject object at 0x17a9fcac0>, 'qelectron': None, 'error': None, 'sublattice_result': None, 'stdout': None, 'stderr': None}
{'node_id': 1, 'node_name': ':parameter:15000', 'start_time': datetime.datetime(2024, 10, 25, 11, 32, 37, 960480), 'end_time': datetime.datetime(2024, 10, 25, 11, 32, 37, 960480), 'status': Status(STATUS=<StatusEnum.COMPLETED: 'COMPLETED'>), 'output': <covalent.TransportableObject object at 0x17a9ff190>, 'qelectron': None, 'error': None, 'sublattice_result': None, 'stdout': None, 'stderr': None}
{'node_id': 2, 'node_name': 'get_eig', 'start_time': datetime.datetime(2024, 10, 25, 11, 32, 38, 66492), 'end_time': datetime.datetime(2024, 10, 25, 11, 36, 47, 818610), 'status': Status(STATUS=<StatusEnum.COMPLE

In [5]:
x = output.get_all_node_results()[0]
print(x)
y = x['output']
y.get_serialized()

{'node_id': 0, 'node_name': 'get_eig', 'start_time': datetime.datetime(2024, 10, 25, 11, 32, 38, 31536), 'end_time': datetime.datetime(2024, 10, 25, 11, 36, 47, 796387), 'status': Status(STATUS=<StatusEnum.COMPLETED: 'COMPLETED'>), 'output': <covalent.TransportableObject object at 0x17a9fcac0>, 'qelectron': None, 'error': None, 'sublattice_result': None, 'stdout': None, 'stderr': None}


'...'

In [14]:
xo._object

'...'

In [15]:
result1 = ct.get_result(dispatch_id=dispatch_id, wait=True)
print(result1.status)

COMPLETED


In [18]:
result.result == result1.result

True

In [33]:
import numpy as np

n = 1000

def eig_hist(n=1000, bin_range=(-13, 13)):
    mat = np.random.random((n, n))
    mat -= 0.5
    mat = mat + mat.T
    mat *= 0.5
    e = np.linalg.eigvalsh(mat)
    #print(e.min(), e.max())
    b, x = np.histogram(e, bins=1000, range=bin_range)
    return (b, x)
    

In [34]:
import covalent_ssh_plugin as ct_ssh

In [35]:
import threading

In [37]:
threads = [threading.Thread() for i in range(10)]

In [38]:
x = threads[0]

In [41]:
x.start()

In [42]:
x.join()

In [43]:
%timeit eig_hist()

589 ms ± 71.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
eig_hist()

(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
        1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 2, 1, 0,
        0, 0, 2, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 1, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 0, 2, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
        1, 1, 1, 0, 2, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 2, 0, 1, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1, 1, 0, 2, 1,
        0, 1, 0, 1, 1, 1, 0, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 0,
        2, 0, 2, 1, 0, 1, 2, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 1,
        0, 2, 0, 2, 1, 0, 2, 0, 1, 2, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 0, 1,
        2, 1, 1, 2, 3, 0, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 0,
        1, 1, 1, 2, 1, 1, 1, 2, 1, 0, 2, 1, 2, 1, 1, 2, 1, 1, 1, 0, 3, 1,
        1, 1, 2, 0, 1, 1, 2, 1, 0, 1, 

In [16]:
import socket
host = socket.gethostname()
print(host)

618-rajarshi
